In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Cenário II

O segundo entregável consiste na transformação de dados disponíveis em arquivo Json para o formato de dataframe, algo comum no dia a dia da empresa. Após transformar esse Json em dataframe é possível perceber que a coluna "item_list" está como dicionário. Seu gestor pediu dois pontos de atenção nessa tarefa:

Expandir a coluna num mesmo dataframe;
Normalizar os itens dessa coluna de dicionário e dividí-los em dois dataframes separados, seguindo o modelo relacional.

In [2]:
df = pd.read_json('dados/data.json')
print('Shape:', df.shape)

Shape: (3, 6)


In [3]:
df.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ItemList
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"[{'ProductName': 'Rice', 'Value': 35.55, 'Quantity': 2}, {'ProductName': 'Flour', 'Value': 11.55, 'Quantity': 5}, {'ProductName': 'Bean', 'Value': 27.15, 'Quantity': 7}]"
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,"[{'ProductName': 'Tomate', 'Value': 12.25, 'Quantity': 10}, {'ProductName': 'Pasta', 'Value': 7.55, 'Quantity': 5}]"
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,"[{'ProductName': 'Beer', 'Value': 9.0, 'Quantity': 6}, {'ProductName': 'French fries', 'Value': 10.99, 'Quantity': 2}, {'ProductName': 'Ice cream', 'Value': 27.15, 'Quantity': 1}]"


### Expandir coluna no mesmo dataframe

In [4]:
df = df.explode('ItemList').reset_index(drop=True)   # Cada item da lista vai para uma linha

In [5]:
df.head(3)

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ItemList
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"{'ProductName': 'Rice', 'Value': 35.55, 'Quantity': 2}"
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"{'ProductName': 'Flour', 'Value': 11.55, 'Quantity': 5}"
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"{'ProductName': 'Bean', 'Value': 27.15, 'Quantity': 7}"


### Normalizar itens

In [6]:
df1 = pd.json_normalize(df['ItemList'])  # Cada item da lista vira uma coluna
df = pd.concat([df, df1], axis=1)        # Adiciona colunas no dataframe original
df = df.drop(columns=['ItemList'])       # Remove coluna ItemList

In [7]:
df.head(3)

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ProductName,Value,Quantity
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Flour,11.55,5
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Bean,27.15,7


### Dividir em dois dataframes (modelo relacional)

In [8]:
# Utiliza "NFeNumber" e "NFeID" como chave composta
df1 = df[['CreateDate', 'EmissionDate', 'Discount', 'NFeNumber', 'NFeID']].drop_duplicates(ignore_index=True)
df2 = df[['NFeNumber', 'NFeID', 'ProductName', 'Value', 'Quantity']]

In [9]:
df1.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3


In [10]:
df2.head()

,NFeNumber,NFeID,ProductName,Value,Quantity
0,501,1,Rice,35.55,2
1,501,1,Flour,11.55,5
2,501,1,Bean,27.15,7
3,502,2,Tomate,12.25,10
4,502,2,Pasta,7.55,5
